<a href="https://colab.research.google.com/github/KorneliuszDyszczynski/ColorizingPhotosNeuralNetwork/blob/master/ColorizingImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r ../Humans2/*

rm: cannot remove '../Humans2/*': No such file or directory


In [2]:

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.dataset_download_files('ashwingupta3012/human-faces')

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
import keras
import numpy as np
import keras.preprocessing.image
from keras.preprocessing.image import img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from keras.layers import Conv2D, UpSampling2D, InputLayer
from keras.models import Sequential

from PIL import Image
import os, sys
from matplotlib.pyplot import imsave

In [4]:
!unzip human-faces.zip

Archive:  human-faces.zip
replace Humans/1 (1).jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [6]:
os.chdir('/content/Humans/') 
path='/content/Humans/'
path2='/content/Humans2/'
path3='/content/'
COUNT =0
w = 200
h = 200
stop=10;

# for f in os.listdir():
#     f_name, f_ext = os.path.splitext(f)
#     f_name = "face" + str(COUNT) 
#     COUNT = COUNT + 1
#     new_name = '{}{}'.format(f_name, f_ext)
#     os.rename(f, new_name)

for f in os.listdir():
    COUNT = COUNT + 1
    if(COUNT<stop):
      im = Image.open(path+f)
      im = im.resize((w, h))
      im = im.convert('RGB')
      im.save(path2+"resized"+f)

os.chdir('/content/Humans2/') 

images = [Image.open(f) for f in os.listdir()]
widths, heights = zip(*(i.size for i in images))

total_width = sum(widths)
max_height = max(heights)

new_im = Image.new('RGB', (total_width, max_height))

x_offset = 0
for im in images:
  new_im.paste(im, (x_offset,0))
  x_offset += im.size[0]

new_im.save(path3+'test.jpg')


image = img_to_array(load_img(path3+'test.jpg'))
image = np.array(image, dtype=float)


OSError: ignored

In [ ]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y = Y / 128
witdthvar=(stop-1)*200;
X = X.reshape(1, 200, witdthvar, 1)
Y = Y.reshape(1, 200, witdthvar, 2)

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

model.compile(optimizer='rmsprop',loss='mse')

In [ ]:
model.fit(x=X, y=Y, batch_size=1, epochs=100)
print(model.evaluate(X, Y, batch_size=1))

In [ ]:
color_me = []
for filename in os.listdir('/Test/'):
        color_me.append(img_to_array(load_img('/Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))




In [ ]:
output = model.predict(color_me)
output = output * 128
canvas = np.zeros((400, 400, 3))
canvas[:,:,0] = color_me[0][:,:,0]
canvas[:,:,1:] = output[0]
imsave(path3+"img_result.png", lab2rgb(canvas))
imsave(path3+"img_gray_scale.png", rgb2gray(lab2rgb(canvas)))
